In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
dataset = pd.read_csv("Data.csv", delimiter=",")
#del dataset['commitid']
#del dataset['revid']
#del dataset['assignment']
#del dataset['airportevent']
#del dataset['affectedairports']
#del dataset['paxcapacity']
#del dataset['duplicate']

del dataset['commitid']
del dataset['revid']

del dataset['paxcapacity']
del dataset['duplicate']



dataset.shape

In [ ]:
dataset.head()

In [ ]:
X= dataset.iloc[:,0:12]
Y= dataset.iloc[:,12:13]
X = np.array(X)
Y = np.array(Y)
Y

In [ ]:
from sklearn.model_selection import KFold
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)
kf = KFold(n_splits=10)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    

In [ ]:
param_grid = {
    'activation': ["identity", "logistic", "tanh", "relu"],
    'solver': ["lbfgs", "sgd", "adam"],
    'batch_size': [100, 250, 500, 750, 1000, 1500],
    'learning_rate': ["constant", "invscaling","adaptive"],
    'max_iter': [1000, 5000, 10000, 15000, 20000]
}
grid = GridSearchCV(SVC(probability =True),param_grid,refit=True,verbose=10)
grid.fit(X_train,y_train)


In [ ]:
print(grid.best_params_) 
  
#printing how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

In [ ]:
#for cvscores

visualiser = CVScores(clf, cv=kf, scoring='f1_weighted')
visualiser.fit(X_train, y_train)        # Fit the data to the visualizer
visualiser.poof() 

In [ ]:
#using the best parameters to fit the model 

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_test

In [ ]:

from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,), activation = 'tanh', max_iter=15000, alpha=0.0001, 
                    solver='lbfgs', batch_size = 500, warm_start= False, verbose=500, tol=1e-5, random_state=1,
                    )
history = mlp.fit(X_train, y_train)
print(mlp)


In [ ]:
from sklearn.metrics import accuracy_score 
predictions = mlp.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))



In [ ]:
predictions

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print("")
print("###Confusion Matrix###")
print(confusion_matrix(y_test, predictions))
print("")
print("###Classification Report###                                  MLP")
print(classification_report(predictions,y_test))

In [ ]:

import matplotlib.pyplot as plt
import scikitplot as skplt

skplt.metrics.plot_confusion_matrix(y_test, predictions, normalize=True)
print("MLP")
print("")
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("")
print(confusion_matrix(y_test, predictions))

In [ ]:
print("Accuracy: %.2f%%" % (accuracy * 100.0))
from yellowbrick.classifier import ClassificationReport
visualizer = ClassificationReport(mlp, support=True)

visualizer.fit(X_train, y_train)  # Fit the visualizer and the model
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.poof()


In [ ]:

# roc curve and auc

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
probs = mlp.predict_proba(X_test)
preds = probs[:,1]

In [ ]:
preds

In [ ]:
auc = roc_auc_score(y_test, preds)
print('AUC: %0.9f' % auc)
fpr, tpr, thresholds = roc_curve(y_test, preds)
pyplot.title("MLP")
pyplot.ylabel('True Positive Rate')
pyplot.xlabel('False Positive Rate')
pyplot.plot([0, 1], [0, 1], 'r--')
pyplot.plot(fpr, tpr, 'b', label = 'AUC = %0.9f' % auc)
pyplot.legend(loc = 'lower right')
pyplot.show()